
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [2]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [3]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

Cloning into 'tf-levenberg-marquardt'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 53 (delta 31), reused 29 (delta 14), pack-reused 0
Receiving objects: 100% (53/53), 32.99 KiB | 4.71 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [4]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y3", label="oxigenio")

/tmp/ipykernel_6220/160627683.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-29 00:18:33.027347: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 00:18:33.490238: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 00:18:33.490291: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 00:18:33.517458: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 00:18:33.580037: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 00:18:33.581133: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [6]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [7]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [8]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [9]:
tester.Loop(sort_by='r2',
            boundarie=-0.8,
            hidden_sizes = [[21, 10], [40, 15]],
            regularizers=[ 0.1,0.02],
            learning_rate=[ 0.1,0.01])

Testando combinacao1: Hidden Size=[21, 10], regularizer=0.1, learning_rate=0.1
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  1.716189360000044
Stopped at epoch:  11
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.33728082000016
Stopped at epoch:  7
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.25310826200006886
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.24801844900002834
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.44615387199996803
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.25722005300008277
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 7 +++++++++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_0_0,-0.065394,-0.117343,-0.088576,-0.095792,1.179078,0.678352,2.172743,1.381595
model_1_0_1,-0.043445,-0.207379,-0.034596,-0.078531,1.154786,0.733013,2.065001,1.359831
model_1_0_17,-0.028339,-0.152953,-0.158336,-0.156839,1.138068,0.699970,2.311980,1.458563
model_1_0_22,-0.028259,-0.146037,-0.160499,-0.156687,1.137980,0.695772,2.316298,1.458372
model_1_0_23,-0.028259,-0.146037,-0.160499,-0.156687,1.137980,0.695772,2.316298,1.458372
model_1_0_21,-0.028136,-0.145576,-0.160175,-0.156329,1.137843,0.695492,2.315652,1.457920
model_1_0_20,-0.027957,-0.145097,-0.159641,-0.155808,1.137646,0.695201,2.314584,1.457264
model_1_0_18,-0.027831,-0.145772,-0.158889,-0.155421,1.137506,0.695611,2.313085,1.456775
model_1_0_19,-0.027821,-0.144908,-0.159168,-0.155408,1.137495,0.695086,2.313641,1.456759
model_1_0_16,-0.026974,-0.168951,-0.147363,-0.152743,1.136557,0.709683,2.290079,1.453399


DataFrame salvo em ../results/metrics_1_0
+++++++++++ [1_1] | 1 ++++++++++++++++++
Elapsed time:  0.03337386900011552
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_1] | 2 ++++++++++++++++++
Elapsed time:  0.028566434999902413
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_1] | 3 ++++++++++++++++++
Elapsed time:  0.0286474739998539
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_1] | 4 ++++++++++++++++++
Elapsed time:  0.04874345100006394
Stopped at epoch:  0
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_1] | 5 ++++++++++++++++++
Elapsed time:  0.045488605000173266
Stopped at epoch:  0
1/1 [==============================] - 0s 27ms/step
+++++++++++ [1_1] | 6 ++++++++++++++++++
Elapsed time:  0.04317029300000286
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_1] | 7 ++++++++++++++++++
Elapsed time:  0.02943

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_1_0,-0.028259,0.045116,-1.096956,-0.209318,1.13798,0.97969,1.359299,1.158326
model_1_1_22,-0.028259,0.045116,-1.096956,-0.209318,1.13798,0.97969,1.359299,1.158326
model_1_1_21,-0.028259,0.045116,-1.096956,-0.209318,1.13798,0.97969,1.359299,1.158326
model_1_1_20,-0.028259,0.045116,-1.096956,-0.209318,1.13798,0.97969,1.359299,1.158326
model_1_1_19,-0.028259,0.045116,-1.096956,-0.209318,1.13798,0.97969,1.359299,1.158326
model_1_1_18,-0.028259,0.045116,-1.096956,-0.209318,1.13798,0.97969,1.359299,1.158326
model_1_1_17,-0.028259,0.045116,-1.096956,-0.209318,1.13798,0.97969,1.359299,1.158326
model_1_1_16,-0.028259,0.045116,-1.096956,-0.209318,1.13798,0.97969,1.359299,1.158326
model_1_1_15,-0.028259,0.045116,-1.096956,-0.209318,1.13798,0.97969,1.359299,1.158326
model_1_1_14,-0.028259,0.045116,-1.096956,-0.209318,1.13798,0.97969,1.359299,1.158326


DataFrame salvo em ../results/metrics_1_1
+++++++++++ [1_2] | 1 ++++++++++++++++++
Elapsed time:  0.18512283400013985
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_2] | 2 ++++++++++++++++++
Elapsed time:  0.22040620200004923
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_2] | 3 ++++++++++++++++++
Elapsed time:  0.3451230220000525
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_2] | 4 ++++++++++++++++++
Elapsed time:  0.27794638099999247
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_2] | 5 ++++++++++++++++++
Elapsed time:  0.5029787420000957
Stopped at epoch:  12
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_2] | 6 ++++++++++++++++++
Elapsed time:  0.2703195499998401
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [1_2] | 7 ++++++++++++++++++
Elapsed time:  0.26188666

...

DataFrame salvo em ../results/metrics_2_7
+++++++++++ [2_8] | 1 ++++++++++++++++++
Elapsed time:  0.22879503099989051
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [2_8] | 2 ++++++++++++++++++
Elapsed time:  0.2607480540000324
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [2_8] | 3 ++++++++++++++++++
Elapsed time:  0.3922351250000702
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [2_8] | 4 ++++++++++++++++++
Elapsed time:  0.17237755199994353
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [2_8] | 5 ++++++++++++++++++
Elapsed time:  0.17817317300000468
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [2_8] | 6 ++++++++++++++++++
Elapsed time:  0.2468894179999097
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [2_8] | 7 ++++++++++++++++++
Elapsed time:  0.171754613

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_8_24,0.898698,0.617786,0.830493,0.791462,0.112112,0.207300,0.253811,0.229187
model_2_8_23,0.898952,0.620694,0.834590,0.794848,0.111831,0.205723,0.247677,0.225466
model_2_8_22,0.899207,0.624273,0.838600,0.798355,0.111548,0.203781,0.241672,0.221612
model_2_8_21,0.899442,0.627805,0.842673,0.801888,0.111288,0.201866,0.235574,0.217729
model_2_8_20,0.899717,0.631362,0.846884,0.805517,0.110983,0.199937,0.229269,0.213740
model_2_8_19,0.900060,0.635496,0.851169,0.809345,0.110604,0.197695,0.222853,0.209534
model_2_8_18,0.900421,0.639241,0.855862,0.813332,0.110205,0.195663,0.215826,0.205151
model_2_8_17,0.900752,0.643315,0.860503,0.817372,0.109839,0.193454,0.208875,0.200711
model_2_8_16,0.901103,0.647669,0.865038,0.821417,0.109450,0.191092,0.202086,0.196265
model_2_8_15,0.901454,0.651439,0.869898,0.825518,0.109061,0.189047,0.194808,0.191758


DataFrame salvo em ../results/metrics_2_8
+++++++++++ [2_9] | 1 ++++++++++++++++++
Elapsed time:  0.24294414399992093
Stopped at epoch:  8
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_9] | 2 ++++++++++++++++++
Elapsed time:  0.21379049500001202
Stopped at epoch:  7
1/1 [==============================] - 0s 16ms/step
+++++++++++ [2_9] | 3 ++++++++++++++++++
Elapsed time:  0.19574641499980316
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [2_9] | 4 ++++++++++++++++++
Elapsed time:  0.21764234199986277
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [2_9] | 5 ++++++++++++++++++
Elapsed time:  0.23541253300004428
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_9] | 6 ++++++++++++++++++
Elapsed time:  0.20744661499998074
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_9] | 7 ++++++++++++++++++
Elapsed time:  0.169307

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_9_24,0.898431,0.747186,0.912963,0.839817,0.112406,0.389339,0.145797,0.274731
model_2_9_23,0.899269,0.751484,0.914854,0.842729,0.111480,0.382719,0.142628,0.269735
model_2_9_22,0.900180,0.756072,0.916840,0.845823,0.110471,0.375654,0.139302,0.264429
model_2_9_21,0.901043,0.760577,0.918733,0.848835,0.109516,0.368715,0.136131,0.259264
model_2_9_20,0.901917,0.765228,0.920572,0.851891,0.108549,0.361553,0.133051,0.254022
model_2_9_19,0.902795,0.769952,0.922405,0.854979,0.107578,0.354277,0.129981,0.248726
model_2_9_18,0.903562,0.774407,0.924077,0.857865,0.106728,0.347417,0.127179,0.243776
model_2_9_0,0.903672,0.822037,0.939916,0.887787,0.106606,0.274066,0.100648,0.192458
model_2_9_17,0.904343,0.778994,0.925720,0.860801,0.105864,0.340353,0.124427,0.238741
model_2_9_16,0.905119,0.783638,0.927363,0.863764,0.105005,0.333201,0.121675,0.233659


DataFrame salvo em ../results/metrics_2_9
Testando combinacao3: Hidden Size=[21, 10], regularizer=0.02, learning_rate=0.1
+++++++++++ [3_0] | 1 ++++++++++++++++++
Elapsed time:  1.4664084679998268
Stopped at epoch:  28
1/1 [==============================] - 0s 15ms/step
+++++++++++ [3_0] | 2 ++++++++++++++++++
Elapsed time:  0.3487376289999702
Stopped at epoch:  9
1/1 [==============================] - 0s 14ms/step
+++++++++++ [3_0] | 3 ++++++++++++++++++
Elapsed time:  0.4582192650000252
Stopped at epoch:  12
1/1 [==============================] - 0s 16ms/step
+++++++++++ [3_0] | 4 ++++++++++++++++++
Elapsed time:  0.2496439700000792
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [3_0] | 5 ++++++++++++++++++
Elapsed time:  0.324625400000059
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [3_0] | 6 ++++++++++++++++++
Elapsed time:  0.5127152299999125
Stopped at epoch:  14
1/1 [==============================] - 0